# 네이버 블로그 크롤링 (네이버 API 이용)


## 들어가며...



## 1) 크롤링? 파싱?

### 1-1) 크롤링

- '웹 크롤러'라는 단어에서 유래되었음.
- 크롤러란 조직적, 자동화된 방법으로 WWW를 탐색하는 컴퓨터 프로그램.
- 크롤링은 크롤러가 하는 작업을 부르는 말.
- 여러 인터넷 사이트의 페이지(문서,html 등)를 수집해서 분류하는 것.
- 대체로 찾아낸 데이터를 저장한 후 쉽게 찾을 수 있게 인덱싱 수행.

### 1-2) 파싱

- 파싱이란 어떤 페이지(문서, html 등)에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출하여 정보를 가공하는 것.
- 파싱이란 일련의 문자열을 의미있는 '토큰'으로 분해하고 이들로 이루어진 '파스 트리'를 만드는 과정.
- 입력 토큰에 내제된 자료 구조를 빌드하고 문법을 검사하는 역할을 함.

## 2) 애플리케이션 등록 ( API 이용신청)

![애플리케이션 등록_1](https://user-images.githubusercontent.com/51112316/61574953-2f655d80-ab01-11e9-8b8f-cc74183302da.jpg)
![애플리케이션 등록_2](https://user-images.githubusercontent.com/51112316/61574954-31c7b780-ab01-11e9-9274-e1d237547d94.jpg)

### 2-1) 네이버API  예제 사용

- 네이버개발자 -> Products -> 서비스API -> 검색 -> 개발 가이드 보기 -> 0.API호출예제 -> Python

#### [API호출예제](https://developers.naver.com/docs/search/blog/)

~~~python
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.net.HttpURLConnection;
import java.net.URL;
import java.net.URLEncoder;

public class APIExamSearchBlog {

    public static void main(String[] args) {
        String clientId = "YOUR_CLIENT_ID";//애플리케이션 클라이언트 아이디값";
        String clientSecret = "YOUR_CLIENT_SECRET";//애플리케이션 클라이언트 시크릿값";
        try {
            String text = URLEncoder.encode("그린팩토리", "UTF-8");
            String apiURL = "https://openapi.naver.com/v1/search/blog?query="+ text; // json 결과
            //String apiURL = "https://openapi.naver.com/v1/search/blog.xml?query="+ text; // xml 결과
            URL url = new URL(apiURL);
            HttpURLConnection con = (HttpURLConnection)url.openConnection();
            con.setRequestMethod("GET");
            con.setRequestProperty("X-Naver-Client-Id", clientId);
            con.setRequestProperty("X-Naver-Client-Secret", clientSecret);
            int responseCode = con.getResponseCode();
            BufferedReader br;
            if(responseCode==200) { // 정상 호출
                br = new BufferedReader(new InputStreamReader(con.getInputStream()));
            } else {  // 에러 발생
                br = new BufferedReader(new InputStreamReader(con.getErrorStream()));
            }
            String inputLine;
            StringBuffer response = new StringBuffer();
            while ((inputLine = br.readLine()) != null) {
                response.append(inputLine);
            }
            br.close();
            System.out.println(response.toString());
        } catch (Exception e) {
            System.out.println(e);
        }
    }
}
~~~

#### 문제 발생 :  

#### 네이버 블로그 형식 맞춰 긁어왔음.
                  
#### 사실은 요약된 내용.

#### 모든 내용을 다 보고싶으면 개발이 필요.

# 네이버 블로그 크롤링

In [8]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

naver_Client_id = "E4zDnlSmVAnJ7dvIOUUx"
naver_Client_secret = "lLxEw7DWbB"

In [9]:
def get_blog_count(query,display) :
    encode_query=urllib.parse.quote(query)
    search_url="https://openapi.naver.com/v1/search/blog?query="+ encode_query
    # 위의 https://~ 는 따온 API호출예제에 있음.
    request=urllib.request.Request(search_url)
    # 이렇게 하면 내가 만들었던 url 형태로 요청한다.
    # 그러나 이렇게 하면 API로부터 거부당한다.
    # 왜냐하면 ID, Secret을 주지 않았기 때문에 아래와 같이 이것을 입력해준다.
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    # code를 받아와서 웹에서 정상적으로 접속했을 때 200번을 준다.
    if response_code is 200 :
        response_body = response.read()  # 실제로 읽어 들인다.
        response_body_dict = json.loads(response_body.decode('utf-8'))
        # 네이버에서 넘겨줄 때 json으로 넘겨주는데 이것을 전부 '딕셔너리'로 저장한다.
        
        print('Last build date :' + str(response_body_dict['lastBuildDate']))
        # json으로 넘겨받을 것들 중에서 최종날짜를 출력
        print('Total :' + str(response_body_dict['total']))  # 관측개수
        print('Start :' + str(response_body_dict['start']))
        print('Display :' + str(response_body_dict['display']))
        #이렇게 넘어온 것을 다 출력해본다.
        
        # 검색된 결과가 0 일 경우        
        if response_body_dict['total']== 0:
            blog_count = 0
        else :
            blog_total=math.ceil(response_body_dict['total']/int(display))
            # 위에서 한 번에 10개씩 출력한다고 했으니 나눠주어서 개수를 맞춰준다.
        
            if blog_total >= 1000 :
                blog_count = 1000
                # 최대 1000개로 제한이 있으니 1000개를 넘었을 때는 1000개로 지정한다.
            else : 
                blog_count = blog_total
        
            print('Blog total :' + str(blog_total))
            print('Blog count : ' + str(blog_count))
        
        return blog_count

- 블로그의 모든 내용을 가져오는 함수[(API사용예제)는 샘플 내용만 가져온다.]



In [10]:
def get_blog_post(query,display,start_index,sort):
    global no, fs
    #get_blog_count(query,display)를 정의할 때 사용했던 것과 동일하게 사용(아래 7문장)
    encode_query = urllib.parse.quote(query)
    # 동일하게 사용하나 search_url에는 옵션을 조금 더 추가해 준다.
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query +"&display=" + str(display) + "&start=" + str(start_index) + "sort=" + sort
    request=urllib.request.Request(search_url)
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    #print(response_code) 이분은 윗부분까지 해서 접속이 잘되었는지를 확인하는 것.
        
    if response_code is 200:
        response_body=response.read()
        response_body_dict=json.loads(response_body.decode('utf-8'))
        # json이 해당하는 블로그가 여러개 나오면 전부 다 하나의 큰 덩어리로 주는데
        # 나눠서 하나씩 접근 하기 위해 아래의 코드를 짜줌.
        for item_index in range(0,len(response_body_dict['items'])):
            # 혹시 태그부분이 따라올 수 있기 때문에
            # 그것을 지우고 사용하기 위해  re(regular expression)으로 제거를 해준다.
            # 즉 태그를 없애주는 정규표현식을 이용한다.
            try :
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['title'])
                #또한 주소가 따라오는데 이건 필요 없는 것이다.그러므로 없애준다.
                link = response_body_dict['items'][item_index]['link'].replace("amp;","")
                # description에도 마찬가지로 태그가 따라올 수 있기 때문에 제거해준다.
                description = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                #  포스트(개시글)의 주소가 아니라 블로그 자체의 링크를 가져옴.
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                #아래처럼 변환해서 가져와도 되고 그냥 주는대로 가져와도 된다.
                #post_date = datetime.datetime.strptime(response_body_dict[['item'][item_index]['postdate'],"%Y%m%d").strtime("%Y.%m.%d")
                
               # 이제 json에서 오는 결과를 하나씩 items에 다 들어가 있다. 그것을 이제 다 끄집어서 가져온다.
                no +=1
                print("-------------------------------------------------") #각각의 게시글을 구분시킴
                print("#"+ str(no))  #몇번째 게시글인지 출력
                print("Title :"+ title)
                print("Link :"+ link)
                print("Description :"+ description)
                print("Blogger Name :"+ blogger_name)
                print("Blogger Link :"+ blogger_link)
                print("Post Date :"+ post_date)
                
                                                                              
                post_code = requests.get(link)
                #  저 정보들은 사실 네이버 오픈 API에서 제공해주는 정보다.
                # 그런데 우리가 필요한 것은 실제 블로그에 포스트 된 내용이 필요하기 때문에
                # 뷰티풀숲을 이용해 가져온다. 
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text,'lxml')
                
                # 네이버 블로그의 문제점 : 마우스를 긁어오지 못하게 iframe으로 만들어 놓은게 있다.
                #그래서 iframe#mainFrame 이라는 html 부분을 들어가서
                # 그 부분의 실제 post url을 뽑을 수 있다.
                # 그냥 가져와선 안되고 그 url에서 소스부분(src)만 가져온다.
                # 그래서 그걸로 접속해서 실제적인 블로그 포스트에 대한 내용을 긁어오기 때문에
                # 여기에 다시 한번 "blog_post_url"이라는 주소로 접근을 한다.
                # 그리고 그것들에 대한 정보를 가져온다.
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com"+mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text,'lxml')
                    
                    # 블로그 전체의 여러가지 메뉴들도 있고 그런 것 제외하고 postViewArea라는 부분에
                    # 실제 포스트 내용이 들어있다. 그래서 여기로 선택한다. 이게 진짜 블로그 포스트의 컨텐트다. 
                    for blog_post_content in blog_post_soup.select("div#postViewArea"):
                        blog_post_content_text = blog_post_content.get_text()
                        blog_post_full_contents = str(blog_post_content_text) #컨텐트를 string으로 캐스팅해서 가져온다.
                        blog_post_full_contents = blog_post_full_contents.replace("\n\n",'\n') 
                        #요약된 description 만 주는게 아니라 이젠 진짜로 된 포스트를 준다.
                        #print("blog_post_contents : " + blog_post_full_contents+"\n")
                        #전체 내용을 print 하면 너무 많기 때문에 위의 코드로 하지않고 파일로 저장한다.
                        #이를 통해 이후에 데이터 분석 등에 사용한다.
                        fs.write(blog_post_full_contents+'\n')
                        fs.write("-----------------------------------")
            #만약에 못할 경우 그냥 넘어가는 코드.
            except: 
                item_index+=1
                print("■■■■■■저장할 수 없음.■■■■■■")

In [17]:
if __name__=='__main__' :
    no = 0           # 몇 개의 포스트를 저장했는지 세기 위한 index
    query = "다나스" # 검색을 원하는 문자열로써 UTF-8로 인코딩한다.
    display = 10     # 검색 결과 출력 건수 지정, 10(기본값), 100(최대값)
    start = 1        # 검색 시작 위치로 최대 1000까지 가능
    sort = 'date'    # 정렬 옵션 : sim(유사도순, 기본값), date(날짜순)
    
    #블로그 콘텐츠의 한글 저장을 위해 encoding='utf-8'으로 설정.
    fs = open(query + ".txt",'a',encoding='utf-8')
    
    blog_count = get_blog_count(query,display)
    for start_index in range(start,blog_count + 1, display):
        get_blog_post(query,display,start_index,sort)  
    
       
    fs.close()

Last build date :Mon, 22 Jul 2019 10:46:50 +0900
Total :18853
Start :1
Display :10
Blog total :1886
Blog count : 1000
-------------------------------------------------
#1
Title :태풍 다나스 제주도 다낫쓰~(백약이오름 코스모스 메렁, 뒷북... 
Link :https://blog.naver.com/9unique?Redirect=Log&logNo=221591516842
Description :태풍 다나스 제주도는 다낫쓰~ㅎ 백약이오름 옆 황화코스모스 메렁이니까 일부러 가지마삼요 (이런곳... 태풍 다나스 소식으로 뉴스와 기사에 대서특필이었다 특히 항상 그래왔듯이 제주도 먼 남쪽 바다 풍랑을... 
Blogger Name :유니크의 제주살이
Blogger Link :https://blog.naver.com/9unique
Post Date :20190721
-------------------------------------------------
#2
Title :5호 태풍 다나스 경로 유동적 및 한반도 북상 전망 (7/18)
Link :https://blog.naver.com/jungjinkim97?Redirect=Log&logNo=221589139683
Description :현재 5호 태풍 다나스가 북상하고 있습니다. 태풍경로가 다소 오락가락하여 처음에는 서울 관통이었다가 남부지방으로 갔다가 지금은 다시 내륙으로 서편화되는 양상을 보이고 있습니다. 태풍 5호 다나스... 
Blogger Name :서울 기상센터 Weather Center of Seoul
Blogger Link :https://blog.naver.com/jungjinkim97
Post Date :20190718
-------------------------------------------------
#3
Title :포천 붕어낚시 2019년 7월 5호태풍 &q

-------------------------------------------------
#23
Title :태풍 다나스 농장 순찰 옥수수
Link :https://blog.naver.com/eunyboy?Redirect=Log&logNo=221590717320
Description :태풍 다나스의 영향으로 새벽부터 비가 내립니다. 오랜 가뭄으로 갈증이 심했는데 정말 반가운 장맛비입니다. ^^ 반면... 남부지방은 이번 태풍으로 큰 피해가 예상된다고 하네요. 부디 큰 피해가 없기를... 
Blogger Name :허브와 풍뎅이
Blogger Link :https://blog.naver.com/eunyboy
Post Date :20190720
-------------------------------------------------
#24
Title :5호 태풍 다나스 우리 동네 경기도 군포시는 큰 피해 없이... 
Link :https://blog.naver.com/rndsoehd112?Redirect=Log&logNo=221590605248
Description :다나스에 의한 큰 피해 없이 지나간 것 같습니다. 아침에는 비가 조금 내리더니 지금은 햇볕이 보이는 맑은 날씨입니다. 땅도 콘크리트 부분은 말라 있고 흙 부분만 조금 젖어 있는 수준입니다. 5호 태풍 다나스... 
Blogger Name :인생이 즐겁다.
Blogger Link :https://blog.naver.com/rndsoehd112
Post Date :20190720
-------------------------------------------------
#25
Title :태풍 다나스, 베고니아들 모여봐~!
Link :https://blog.naver.com/ksrmg2002?Redirect=Log&logNo=221590527019
Description :태풍 5호 다나스 간접영향으로 강풍이 불고 있어서 문은 꼭 꼭 닫고 창밖으로 내리는 비구경하고 있어요. 밤새 비가 많이 내렸고 바람소리가

-------------------------------------------------
#45
Title :태풍경로 다나스? 아직안와스 내감기도 다나스
Link :https://blog.naver.com/hangilchi?Redirect=Log&logNo=221590533276
Description :딱 걸려쓰 2019년 7월20일 태풍 다나스가 몰려온 부산 거제동 법조타운의 모습입니다. 갱년기 탱자... 태풍 다나스는 오는데 내감기는 아직 다안나스 다나스 다와쓰? 아직 안와쓰?ㅋㅋ 이런 아재 개그만 떠오르는... 
Blogger Name :TAKE ME WITH YOU..나를 데려가 주세요
Blogger Link :https://blog.naver.com/hangilchi
Post Date :20190720
-------------------------------------------------
#46
Title :태풍 다나스와 강낭콩 수확
Link :https://blog.naver.com/kgkeh?Redirect=Log&logNo=221589548107
Description :5호 태풍 다나스는 속도가 그리 빠르지 않아 비 피해가 크다고 한다. 우리나라 기상청의 태풍 진로도와 지금 우리나라와 사이가 좋지 않은 일본의 태풍 진로도가 다르다. 어찌 되었건 우리나라에 다나스의... 
Blogger Name :헬스팜
Blogger Link :https://blog.naver.com/kgkeh
Post Date :20190719
-------------------------------------------------
#47
Title :[카드뉴스] 제5호 태풍 '다나스' 북상 중! 이렇게 대비해주세요
Link :https://blog.naver.com/mopaspr?Redirect=Log&logNo=221589087249
Description :지금 현재 제5호 태풍 '다나스'가 북상 중에 있습니다. 행정안전부가 전해드리는 '태풍 대비요령' 이렇게... #태풍

-------------------------------------------------
#67
Title :태풍5호다나스가 오고 있어 잠이 들지 않아 밤새뜯눈으로 지냈다.
Link :https://blog.naver.com/dora48?Redirect=Log&logNo=221590473652
Description :#태풍5호다나스 밤새 비가 내렸다. 울부부 뜬눈으로 밤을 샜다. 낭군은 아침일찍 하우스로 달려 갔다.... #태풍5호 다나스 제발 조용히 지나가길 빌어본다. 농사짓는 농부들 맘은 매한가지리라 무탈하게 이또한... 
Blogger Name :돌산갓대율마을체험학습장
Blogger Link :https://blog.naver.com/dora48
Post Date :20190720
-------------------------------------------------
#68
Title :태풍 '다나스' 소멸되가는 2019.7.20 오후3시 #학원농장 현황
Link :https://blog.naver.com/bomeh?Redirect=Log&logNo=221590697346
Description :태풍 '다나스'로 온국민이 피해가 없길 기도하며 일기예보를 주시하는 가운데, 오늘 오후 3시에 촬영한 학원농장 메밀꽃밭 입니다. 비는 오락가락해도 다행히 태풍의 영향없이 메밀꽃들은 예정대로 절정돌입... 
Blogger Name :학원농장 경관이야기 bomeh
Blogger Link :https://blog.naver.com/bomeh
Post Date :20190720
-------------------------------------------------
#69
Title :태풍 다나스의 진로에 있는 제주날씨
Link :https://blog.naver.com/logenjjem?Redirect=Log&logNo=221589833506
Description :열대성 폭풍, 제5호 태풍 다나스가 점차 제주로 접근하고 있습니다. 이로 인해 내일... 18일 저녁 

-------------------------------------------------
#89
Title :태풍 다나스. 제주도 조천리 바닷가 마을 현재 모습. 19년... 
Link :https://blog.naver.com/sanghooooo?Redirect=Log&logNo=221590229005
Description :오늘은 태풍 다나스. 밤이다;;; 오전부터 기록적인 폭우로 도로는 맨홀 위로 빗물이 오바이트를 하고... 태풍 다나스 집 앞 모습. 그리고 집 뒷골목 모습. 현재 동풍이 부는 중이다. 소형태풍이라곤 하지만 비가... 
Blogger Name :집宇집宙 아기랑 제주도 숙소. 함덕펜션 그리고 1인실
Blogger Link :https://blog.naver.com/sanghooooo
Post Date :20190719
-------------------------------------------------
#90
Title :부산 해운대 / 태풍 다나스 현재 상황
Link :https://blog.naver.com/kangpoorun?Redirect=Log&logNo=221590507725
Description :저는 지금 부산 해운대에 와있어요 모처럼 금요일 연차를 받아서 해운대 호텔로 호캉스 왔는데 태풍소식이 ㅠㅠ 태풍 다나스가 상륙한다고 해서 예상은 하고 왔지만 비가 이정도로 많이 올줄은.. 아침에... 
Blogger Name :안녕, 나의 soulmate
Blogger Link :https://blog.naver.com/kangpoorun
Post Date :20190720
-------------------------------------------------
#91
Title :태풍 다나스를 느껴보다
Link :https://blog.naver.com/pretty3597?Redirect=Log&logNo=221590974274
Description :태풍 느끼기 태풍 다나스가 어느정도의 간접 영향을 미치나 궁금했다 집안은 이렇게

...

-------------------------------------------------
#909
Title :태풍경로 / 5호 태풍 다나스
Link :https://blog.naver.com/maum8800?Redirect=Log&logNo=221589112520
Description :이번 주 주말엔 제5호 태풍인 ' 다나스 ' 가 한반도를 통과할 거라고 합니다. 그렇다면 태풍이 생기는... ▶ 제5호 태풍 다나스 ◀ 출처.네이버 날씨 출처.네이버 날씨 태풍 다나스는 규모는 소형급 태풍이지만... 
Blogger Name :청주 복대동 마음에꽃한의원
Blogger Link :https://blog.naver.com/maum8800
Post Date :20190718
-------------------------------------------------
#910
Title :태풍 다나스 열대저압부로 소멸…광주·전남 태풍경보 해제
Link :https://blog.naver.com/cwsuek?Redirect=Log&logNo=221590630393
Description :태풍 다나스 열대저압부로 소멸…광주·전남 태풍경보 해제 (광주=연합뉴스) 박철홍 기자 = 제5호 태풍 '다나스'(DANAS)가 20일 정오께 전남 진도군 서쪽 약 50㎞ 부근 해상에서 열대저압부로 약화했다.... 
Blogger Name :너무 좋아하는 건프라모델들
Blogger Link :https://blog.naver.com/cwsuek
Post Date :20190720
-------------------------------------------------
#911
Title :[태풍 경로] 태풍 다나스 경로
Link :https://blog.naver.com/jacobby?Redirect=Log&logNo=221589586968
Description :제5호 태풍 '다나스'가 대한해협이 아닌 한반도를 관통해 동해상으로 빠져나갈 것으로 예측됐다. 다나스가 속도를 내면서 이르면 이날

-------------------------------------------------
#931
Title :[북서태평양] 열대폭풍 다나스 (Tropical Storm 'DANAS') #05
Link :https://blog.naver.com/jaemin0593?Redirect=Log&logNo=221589802956
Description :태풍 정보발표시각 : 2019.07.19 00:00 (UTC) 2019.07.19 09:00 (KST) 명칭 : 열대폭풍 다나스 (Tropical Storm 'DANAS') 위치(JMA) : 29.6N 124E 중국 저장성 저우산시 Lisha Ecological Park 동쪽 약 159km 부근 해상... 
Blogger Name :우리나라 기상, 세계 열대저기압 블로그 (OPEN)
Blogger Link :https://blog.naver.com/jaemin0593
Post Date :20190719
-------------------------------------------------
#932
Title :태풍 다나스(DANAS) 이동경로 어플, 태풍 경보 발령시 대비요령
Link :https://blog.naver.com/jungeumpark?Redirect=Log&logNo=221590306369
Description :태풍 다나스 이동경로 볼 수 있는 어플입니다. earth :: a global map of wind, weather, and ocean conditions... earth.nullschool.net 태풍 다나스 5호 정보입니다. 2019년 태풍 다나스 이동경로를 보면... 
Blogger Name :대단하고 특별하게
Blogger Link :https://blog.naver.com/jungeumpark
Post Date :20190720
-------------------------------------------------
#933
Title :[태풍 다나스 북상] 태풍·호우 국민 행동요령
Li

-------------------------------------------------
#953
Title :제5호 태풍 '다나스' 20일 남부 내륙 관통...집중호우 예상
Link :https://blog.naver.com/hanwoosteel?Redirect=Log&logNo=221589570876
Description :북상중인 태풍 다나스의 영향을 받는 19일 오전 창원시 의창구의 횡단보도를 한 시민이 우산을 쓰고 건너고 있다. 제5호 태풍 다나스(DANAS)가 토요일인 20일 남부 내륙지방을 관통할 전망이다. 이에 따라... 
Blogger Name :지안건축
Blogger Link :https://blog.naver.com/hanwoosteel
Post Date :20190719
-------------------------------------------------
#954
Title :올해 전세계 제출된 태풍 30개! 5호 태풍 다나스 상륙
Link :https://blog.naver.com/4h4l3k?Redirect=Log&logNo=221589379425
Description :오늘은 태풍 다나스에 대해서 알아봤습니다. 태풍 다나스는 어떤 태풍일까요?2019년 올해 전세계 제출된 태풍 이름은 총 30개 정말 다양한 이름의 태풍들이 리스트를 올렸습니다. 2018년은 29개의 태풍이... 
Blogger Name :함께 행복하기~♡
Blogger Link :https://blog.naver.com/4h4l3k
Post Date :20190719
-------------------------------------------------
#955
Title :태풍5호다나스를 보내면서 2019년 태풍이 1호부터 5호까지... 
Link :https://blog.naver.com/sexysiro?Redirect=Log&logNo=221591064213
Description :태풍 다나스때문에 긴급재난문자가 여러번울렸어요 5호 태풍이라고 하니 2019년

-------------------------------------------------
#975
Title :2019년 제5호 태풍 다나스 북상 한반도 영향권?
Link :https://blog.naver.com/budalove?Redirect=Log&logNo=221587936227
Description :제5호 태풍 다나스 북상 중 귀농 첫해 여름. 태풍은 아니었지만 장마때 집중호우로 딸기하우스가 잠긴적이 있어 비가 많이 온다고 하면 걱정부터 앞서거든요. 그런데 이번 태풍은 내일부터 충남지역에도... 
Blogger Name :드림딸기팜♡ 새콤달콤논산 딸기키우기
Blogger Link :https://blog.naver.com/budalove
Post Date :20190717
-------------------------------------------------
#976
Title :태풍 다나스 경로는?...일본 기상청, 21일 열대저기압으로 소멸
Link :https://blog.naver.com/hnewskr?Redirect=Log&logNo=221590454831
Description :기상청의 제5호 태풍 다아스 예상 진행 경로 위성사진 일본기상청의 제5호 태풍 다나스 예상 진행 경로 [현대건강신문=채수정 기자] 제5호 태풍 다나스의 영향으로 전국에 비가 내리고 있는 가운데... 
Blogger Name :현대건강신문
Blogger Link :https://blog.naver.com/hnewskr
Post Date :20190720
-------------------------------------------------
#977
Title :태풍 다나스~! 대비하여 피해를 줄입시다!(더웨이)
Link :https://blog.naver.com/theway_of?Redirect=Log&logNo=221590228228
Description :태풍 다나스에!! 제주와 남부권은 비상 상황인데요 ㅠㅠ 피해 없도록 더웨이는 기도하겠습니다...... 태풍

-------------------------------------------------
#997
Title :제5호 태풍 다나스
Link :https://blog.naver.com/gkstmdal80?Redirect=Log&logNo=221590445417
Description :제주도 및 남부지방은 기록적인 비가 내리고 있다 지리산은 최고700mm까지 남부지방은 100mm까지 태풍 '다나스' 경로 및 태풍 대비 방법 http://naver.me/GV8sdWI0 태풍 '다나스' 경로 및 태풍 대비 방법 제5호... 
Blogger Name :해피한(韓)의 농사일지
Blogger Link :https://blog.naver.com/gkstmdal80
Post Date :20190720
-------------------------------------------------
#998
Title :2019 태풍은  '다나스' 북상중? 전국적으로 장맛비?
Link :https://blog.naver.com/nbnceo?Redirect=Log&logNo=221588218601
Description :2019 태풍은 다나스? 다나스 태풍은 서해를 거쳐서 한반도로 접근한다고 합니다. 오늘 오후 3시 정도에 태풍 다나스는 필리핀 마닐라 북동쪽 480km 부근 해상을 지나면서 시속 15km로 한반도로 달려오고 있는데요.... 
Blogger Name :세일즈렌트카8
Blogger Link :https://blog.naver.com/nbnceo
Post Date :20190717
-------------------------------------------------
#999
Title :태풍  「다나스」  이동경로 ▶▶ 서울 · 수도권 관통한다고??
Link :https://blog.naver.com/modern-yoga?Redirect=Log&logNo=221590082569
Description :제일 큰 피해 입은 제주도 상황 제5호 태풍 다나스 북상에 태풍 영향권에 